In [1]:
import pandas as pd
import numpy as np

import statsmodels.api as sm
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

from energy_consumption.feature_selection.extract import extract_energy_data, extract_all_features
from energy_consumption.help_functions import get_forecast_timestamps, create_submission_frame, lasso_functions

In [2]:
energydata = extract_energy_data.get_data(num_years=0.5)
energydata2 = extract_all_features.get_energy_and_features(energydata, lasso=True)
energydata2

100%|██████████| 26/26 [00:14<00:00,  1.76it/s]


did you update weather and index?
2023-07-17 00:00:00
2024-01-10 12:00:00
len sun_df:  178
len energydata:  4261


,energy_consumption,winter,spring_autumn,saturday,working_day,period1,period2,period3,period4,period5,period6,holiday,abs_log_ret_weekly,sun_hours,tavg,wspd,index,population
date_time,,,,,,,,,,,,,,,,,,
2023-07-19 00:00:00,42.65925,0,0,0,1,0,0,0,0,0,0,0,0.327844,15.834444,17.9,13.7,82.100000,84.581
2023-07-19 01:00:00,40.85850,0,0,0,1,0,1,0,0,0,0,0,0.327844,15.834444,17.9,13.7,82.100000,84.581
2023-07-19 02:00:00,39.76475,0,0,0,1,0,1,0,0,0,0,0,0.327844,15.834444,17.9,13.7,82.100000,84.581
2023-07-19 03:00:00,39.75825,0,0,0,1,0,1,0,0,0,0,0,0.327844,15.834444,17.9,13.7,82.100000,84.581
2023-07-19 04:00:00,40.72475,0,0,0,1,0,1,0,0,0,0,0,0.327844,15.834444,17.9,13.7,82.100000,84.581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-10 08:00:00,70.13225,1,0,0,1,0,0,1,0,0,0,0,0.305194,8.375833,-8.3,8.7,74.473905,84.600
2024-01-10 09:00:00,71.88025,1,0,0,1,0,0,0,0,0,0,0,0.305194,8.375833,-8.3,8.7,74.473905,84.600
2024-01-10 10:00:00,71.80025,1,0,0,1,0,0,0,0,0,1,0,0.305194,8.375833,-8.3,8.7,74.473905,84.600


In [3]:
# important to scale X for lasso regression
energydata3 = extract_all_features.get_energy_and_standardized_features(energydata,
    lasso=True)# [-15124:]

y = energydata3[['energy_consumption']]
X = energydata3.drop(columns=['energy_consumption'])
X = sm.add_constant(X, has_constant="add")

2023-07-17 00:00:00
2024-01-10 12:00:00
len sun_df:  178
len energydata:  4261
                     energy_consumption  winter  spring_autumn  saturday  \
date_time                                                                  
2023-07-19 00:00:00            42.65925       0              0         0   
2023-07-19 01:00:00            40.85850       0              0         0   
2023-07-19 02:00:00            39.76475       0              0         0   
2023-07-19 03:00:00            39.75825       0              0         0   
2023-07-19 04:00:00            40.72475       0              0         0   

                     working_day  period1  period2  period3  period4  period5  \
date_time                                                                       
2023-07-19 00:00:00            1        0        0        0        0        0   
2023-07-19 01:00:00            1        0        1        0        0        0   
2023-07-19 02:00:00            1        0        1        0     

In [5]:
energydata3.info()
X

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4213 entries, 2023-07-19 00:00:00 to 2024-01-10 12:00:00
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   winter              4213 non-null   float64
 1   spring_autumn       4213 non-null   float64
 2   saturday            4213 non-null   float64
 3   working_day         4213 non-null   float64
 4   period1             4213 non-null   float64
 5   period2             4213 non-null   float64
 6   period3             4213 non-null   float64
 7   period4             4213 non-null   float64
 8   period5             4213 non-null   float64
 9   period6             4213 non-null   float64
 10  holiday             4213 non-null   float64
 11  sun_hours           4213 non-null   float64
 12  abs_log_ret_weekly  4213 non-null   float64
 13  tavg                4213 non-null   float64
 14  wspd                4213 non-null   float64
 15  index              

,const,winter,spring_autumn,saturday,working_day,period1,period2,period3,period4,period5,period6,holiday,sun_hours,abs_log_ret_weekly,tavg,wspd,index,population
date_time,,,,,,,,,,,,,,,,,,
2023-07-19 00:00:00,1.0,-0.548004,-1.037493,-0.407513,0.63109,-0.208178,-0.447914,-0.64066,-0.377708,-0.301472,-0.378528,-0.188121,1.785840,-0.970183,0.906469,-0.502588,0.851119,-0.239750
2023-07-19 01:00:00,1.0,-0.548004,-1.037493,-0.407513,0.63109,-0.208178,2.232571,-0.64066,-0.377708,-0.301472,-0.378528,-0.188121,1.785840,-0.970183,0.906469,-0.502588,0.851119,-0.239750
2023-07-19 02:00:00,1.0,-0.548004,-1.037493,-0.407513,0.63109,-0.208178,2.232571,-0.64066,-0.377708,-0.301472,-0.378528,-0.188121,1.785840,-0.970183,0.906469,-0.502588,0.851119,-0.239750
2023-07-19 03:00:00,1.0,-0.548004,-1.037493,-0.407513,0.63109,-0.208178,2.232571,-0.64066,-0.377708,-0.301472,-0.378528,-0.188121,1.785840,-0.970183,0.906469,-0.502588,0.851119,-0.239750
2023-07-19 04:00:00,1.0,-0.548004,-1.037493,-0.407513,0.63109,-0.208178,2.232571,-0.64066,-0.377708,-0.301472,-0.378528,-0.188121,1.785840,-0.970183,0.906469,-0.502588,0.851119,-0.239750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-10 08:00:00,1.0,1.824803,-1.037493,-0.407513,0.63109,-0.208178,-0.447914,1.56089,-0.377708,-0.301472,-0.378528,-0.188121,-1.091446,-1.078411,-2.709049,-1.089822,-1.009243,4.171017
2024-01-10 09:00:00,1.0,1.824803,-1.037493,-0.407513,0.63109,-0.208178,-0.447914,-0.64066,-0.377708,-0.301472,-0.378528,-0.188121,-1.091446,-1.078411,-2.709049,-1.089822,-1.009243,4.171017
2024-01-10 10:00:00,1.0,1.824803,-1.037493,-0.407513,0.63109,-0.208178,-0.447914,-0.64066,-0.377708,-0.301472,2.641811,-0.188121,-1.091446,-1.078411,-2.709049,-1.089822,-1.009243,4.171017


In [20]:
val = float(energydata.iloc[len(energydata)-1][['sun_hours']])
val = energydata.iat[len(energydata)-1, 12]
energydata['sun_hours'] = val 
energydata

C:\Users\Maria\AppData\Local\Temp\ipykernel_16856\4255121795.py:1: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  val = float(energydata.iloc[len(energydata)-1][['sun_hours']])


,winter,spring_autumn,saturday,working_day,period1,period2,period3,period4,period5,period6,holiday,abs_log_ret_weekly,sun_hours,tavg,wspd,index,population,energy_consumption
date_time,,,,,,,,,,,,,,,,,,
2022-03-02 09:00:00,-0.561383,0.99031,-0.408197,0.63236,-0.208491,-0.447262,-0.641617,-0.377947,-0.301509,-0.378004,-0.185675,3.164755,-1.26813,-1.171750,-1.033348,1.318013,1.418894,70.04600
2022-03-02 10:00:00,-0.561383,0.99031,-0.408197,0.63236,-0.208491,-0.447262,-0.641617,-0.377947,-0.301509,2.645476,-0.185675,3.164755,-1.26813,-1.171750,-1.033348,1.318013,1.418894,70.34700
2022-03-02 11:00:00,-0.561383,0.99031,-0.408197,0.63236,-0.208491,-0.447262,-0.641617,-0.377947,-0.301509,2.645476,-0.185675,3.164755,-1.26813,-1.171750,-1.033348,1.318013,1.418894,70.83375
2022-03-02 12:00:00,-0.561383,0.99031,-0.408197,0.63236,-0.208491,-0.447262,-0.641617,-0.377947,-0.301509,2.645476,-0.185675,3.164755,-1.26813,-1.171750,-1.033348,1.318013,1.418894,70.40650
2022-03-02 13:00:00,-0.561383,0.99031,-0.408197,0.63236,4.796369,-0.447262,-0.641617,-0.377947,-0.301509,-0.378004,-0.185675,3.164755,-1.26813,-1.171750,-1.033348,1.318013,1.418894,69.41575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-22 08:00:00,-0.561383,0.99031,-0.408197,0.63236,-0.208491,-0.447262,1.558562,-0.377947,-0.301509,-0.378004,-0.185675,-0.875593,-1.26813,-1.520221,-0.579297,-1.969241,1.766801,65.75125
2023-11-22 09:00:00,-0.561383,0.99031,-0.408197,0.63236,-0.208491,-0.447262,-0.641617,-0.377947,-0.301509,-0.378004,-0.185675,-0.875593,-1.26813,-1.520221,-0.579297,-1.969241,1.766801,67.40950
2023-11-22 10:00:00,-0.561383,0.99031,-0.408197,0.63236,-0.208491,-0.447262,-0.641617,-0.377947,-0.301509,2.645476,-0.185675,-0.875593,-1.26813,-1.520221,-0.579297,-1.969241,1.766801,68.26775


In [14]:
correlation_matrix = X.corr()

# Find pairs of variables with absolute correlation value greater than 0.1
high_corr_pairs = [(i, j) for i in range(correlation_matrix.shape[0])
                   for j in range(i+1, correlation_matrix.shape[1])
                   if abs(correlation_matrix.iloc[i, j]) > 0.1]

# Create a copy of the original predictor matrix
X_pol = X.copy()

# Add polynomial features for selected pairs
for pair in high_corr_pairs:
    col1, col2 = X.columns[pair[0]], X.columns[pair[1]]
    name = f"{col1}_{col2}"
    X_pol[name] = X_pol[col1] * \
        X_pol[col2]
    
# Display the updated predictor matrix with polynomial features
column_set = {'winter_spring_autumn', 'saturday_working_day', 'period1_period3', 'period2_period3',
              'period2_period4', 'period2_period5', 'period2_period6', 'period3_period4', 'period3_period5',
              'period3_period6', 'period4_period5', 'period4_period6', 'period5_period6'}
columns_to_drop = column_set.intersection(set(X_pol.columns))
X_pol = X_pol.drop(columns=list(columns_to_drop))

# add polynomials 
X_pol['tavg_2'] = X_pol['tavg']**2
X_pol['wspd_2'] = X_pol['wspd']**2
X_pol['sun_hours_2'] = X_pol['sun_hours']**2


len(X.columns)

18

Three different feature-selections to train with 
* no polynomials (X)
* selected polynomials (X_pol)
* all possible polynomials (X_all)

In [29]:
from sklearn.linear_model import LassoCV
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt

# Time-based split for time series data
tscv = TimeSeriesSplit(n_splits=20)  # Adjust the number of splits as needed

# Create LassoCV model with time series cross-validation
# let alphas = None to automatically compute best alpha
lasso_cv = LassoCV(cv=tscv,max_iter=10000)

# Fit the model on the scaled data
lasso_cv.fit(X_pol, y)

# Find the best alpha
best_alpha = lasso_cv.alpha_
print(f"Best alpha: {best_alpha}")

c:\Users\Maria\Documents\Setups\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1563: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Best alpha: 0.053099275006647635


In [32]:
lasso_best = Lasso(alpha=best_alpha)
lasso_best.fit(X_pol, y)
print(list(zip(lasso_best.coef_, X_pol)))

[(0.0, 'const'), (0.7235145491854882, 'winter'), (0.0, 'spring_autumn'), (0.9729724146772374, 'saturday'), (5.172729008981154, 'working_day'), (1.9381990563608011, 'period1'), (-2.6766244829940233, 'period2'), (3.744607944114275, 'period3'), (0.5088081158861425, 'period4'), (1.4566235513494905, 'period5'), (3.64982574965883, 'period6'), (-1.8459615002292198, 'holiday'), (-0.11493319996249962, 'abs_log_ret_weekly'), (-1.3512310061451092, 'sun_hours'), (-1.3279293332137587, 'tavg'), (0.6817294262323595, 'wspd'), (1.799202019091538, 'index'), (-0.12806619290452795, 'population'), (0.0, 'winter_sun_hours'), (-0.0, 'winter_tavg'), (-0.0, 'winter_wspd'), (-0.0, 'winter_index'), (-0.0, 'spring_autumn_index'), (0.0, 'abs_log_ret_weekly_population'), (0.0, 'sun_hours_tavg'), (-0.0, 'sun_hours_wspd'), (0.044599083823907454, 'sun_hours_index'), (-0.0, 'tavg_wspd'), (0.21916080867601098, 'index_population'), (0.6253878291192887, 'tavg_2'), (0.06740400889156567, 'wspd_2'), (0.0, 'sun_hours_2')]


(array([ 0.        ,  0.72351455,  0.        ,  0.97297241,  5.17272901,
         1.93819906, -2.67662448,  3.74460794,  0.50880812,  1.45662355,
         3.64982575, -1.8459615 , -0.1149332 , -1.35123101, -1.32792933,
         0.68172943,  1.79920202, -0.12806619,  0.        , -0.        ,
        -0.        , -0.        , -0.        ,  0.        ,  0.        ,
        -0.        ,  0.04459908, -0.        ,  0.21916081,  0.62538783,
         0.06740401,  0.        ]),)

In [ ]:
alphas = np.arange(0.5, 15.1, 0.1)
lasso_cv2 = LassoCV(alphas= alphas, cv=tscv)
lasso_cv2.fit(X_pol, y)

cv2_scores = lasso_cv2.mse_path_
mean_cv2_scores = np.mean(cv2_scores, axis=1)
print(mean_cv2_scores)
print(cv2_scores)

plt.figure(figsize=(10, 6))
plt.plot(alphas, mean_cv2_scores,
         label='Mean Cross-Validated Score', color='black')
plt.xlabel('Alpha')
plt.ylabel('Mean Squared Error')
plt.title('Mean Cross-Validated Performance for Different Alphas')
plt.legend()
plt.show()

In [ ]:
# Plot the scores for each alpha
plt.figure(figsize=(10, 6))
for i in range(cv2_scores.shape[1]):
    plt.plot(alphas, cv2_scores[:, i], label=f'Fold {i + 1}')

plt.xlabel('Alpha')
plt.ylabel('Mean Squared Error')
plt.title('Cross-Validated Performance for Different Alphas')
plt.legend()
plt.show()

Use alpha = 0.0531 for Lasso Regression

In [1]:
import pandas as pd
import numpy as np

import statsmodels.api as sm
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

from energy_consumption.feature_selection.extract import extract_energy_data, extract_all_features
from energy_consumption.help_functions import get_forecast_timestamps, create_submission_frame, lasso_functions

In [2]:
correlated_pairs = [(1, 2), (1, 12), (1, 13),(1, 14),
                   (1, 15),(1, 16), (1, 17), (2, 12),
                   (2, 13),(2, 14), (2, 16), (3, 4),
                   (5, 7),(6, 7),(6, 8),(6, 9),(6, 10),
                   (7, 8),(7, 9),(7, 10),(8, 9),(8, 10),
                   (9, 10),(12, 16),(12, 17),(13, 14),
                   (13, 15),(13, 16),(14, 15),(15, 16),
                   (15, 17),(16, 17)]

column_set = {'winter_spring_autumn', 'saturday_working_day',
                  'period1_period3', 'period2_period3',
                  'period2_period4', 'period2_period5', 'period2_period6',
                  'period3_period4', 'period3_period5',
                  'period3_period6', 'period4_period5',
                  'period4_period6', 'period5_period6'}


def get_interaction_and_pol_terms(X): 
    ''' X needs to be in lasso shape (18 columns)'''

    if len(X.columns) == 18: 

        # Create copy of the original predictor matrix
        X_int_pol = X.copy()

        # Add interaction terms for selected pairs
        for pair in correlated_pairs:
            col1, col2 = X.columns[pair[0]], X.columns[pair[1]]
            name = f"{col1}_{col2}"
            X_int_pol[name] = X_int_pol[col1] * \
                X_int_pol[col2]
            
        # Drop interaction terms for dummies of same category
        columns_to_drop = column_set.intersection(set(X_int_pol.columns))
        X_int_pol = X_int_pol.drop(columns=list(columns_to_drop))

        # Add polynomials
        X_int_pol['tavg_2'] = X_int_pol['tavg']**2
        X_int_pol['wspd_2'] = X_int_pol['wspd']**2
        X_int_pol['sun_hours_2'] = X_int_pol['sun_hours']**2

        return X_int_pol

    else: 
        print('X has wrong number of columns')
        return X

In [21]:
from energy_consumption.feature_selection.clean.impute_outliers import impute_outliers

def get_Lasso_forecasts(energydata=pd.DataFrame(), indexes=[47, 51, 55, 71, 75, 79],
                        quantiles=[0.025, 0.25, 0.5, 0.75, 0.975], periods=100):

    # get standardized features
    energydata = extract_all_features.get_energy_and_standardized_features(
        lasso=True)[-15124:]

    # split df
    y = energydata[['energy_consumption']]
    X = energydata.drop(columns=['energy_consumption'])
    X = sm.add_constant(X, has_constant="add")
    X_int_pol = get_interaction_and_pol_terms(X)

    # fit Lasso Regression with best alpha 
    lasso = Lasso(alpha=0.0531)

    # Fit the model on the scaled data
    lasso.fit(X_int_pol, y)

    # create dataframe to store forecast quantiles
    X_fc = get_forecast_timestamps.forecast_timestamps(
        energydata.index[-1])
    X_fc = extract_all_features.get_energy_and_standardized_features(
        X_fc, lasso=True)
    X_fc = sm.add_constant(X_fc, has_constant='add')

    # approximate sun_hours by last observation 
    last_obs = len(X_int_pol) - 1
    sun_hours_val = float(X_int_pol.iloc[last_obs][['sun_hours']])
    X_fc['sun_hours'] = sun_hours_val
    X_fc_int_pol = get_interaction_and_pol_terms(X_fc)

    print(X_fc_int_pol.head(5))
    print(X_fc_int_pol.info())

    # estimate forecast means
    mean_est = lasso.predict(X_fc_int_pol)[indexes]

    # estimate forecast stds
    mean_est_hist = lasso.predict(X_int_pol)
    variance_est = mean_squared_error(
        y, mean_est_hist)
    indexes_shiftforw = [index+1 for index in indexes]
    forecast_std = np.array([lasso_functions.estimate_forecast_std(
        variance_est, horizon) for horizon in indexes_shiftforw])

    # estimate quantile forecasts
    quantile_forecasts = lasso_functions.get_quantiles(
        mean_est, forecast_std, quantiles)

    # return quantile forecasts in terms of absolute evaluation
    abs_eval = len(quantiles) != 6
    if abs_eval == True:
        horizon = pd.date_range(start=energydata.index[-1] + pd.DateOffset(
            hours=1), periods=periods, freq='H')
        quantile_forecasts.insert(
            0, 'date_time', [horizon[i] for i in indexes])

        return quantile_forecasts

    # else: create submission frame
    else:
        forecast_frame = create_submission_frame.get_frame(
            quantile_forecasts, indexes)
        forecast_frame = forecast_frame.drop(columns={'index'})
        horizon = pd.date_range(start=energydata.index[-1] + pd.DateOffset(
            hours=1), periods=periods, freq='H')
        forecast_frame.insert(
            0, 'date_time', [horizon[i] for i in indexes])

        return forecast_frame



forecast_frame = get_Lasso_forecasts()
forecast_frame

                     const  winter  spring_autumn  saturday  working_day  \
date_time                                                                  
2023-11-22 13:00:00    1.0     0.0            0.0 -0.561951     0.833616   
2023-11-22 14:00:00    1.0     0.0            0.0 -0.561951     0.833616   
2023-11-22 15:00:00    1.0     0.0            0.0 -0.561951     0.833616   
2023-11-22 16:00:00    1.0     0.0            0.0 -0.561951     0.833616   
2023-11-22 17:00:00    1.0     0.0            0.0 -0.561951     0.833616   

                      period1   period2   period3   period4   period5  ...  \
date_time                                                              ...   
2023-11-22 13:00:00  4.358899 -0.436436 -0.670280 -0.369274 -0.294884  ...   
2023-11-22 14:00:00 -0.229416 -0.436436  1.491914 -0.369274 -0.294884  ...   
2023-11-22 15:00:00 -0.229416 -0.436436  1.491914 -0.369274 -0.294884  ...   
2023-11-22 16:00:00 -0.229416 -0.436436  1.491914 -0.369274 -0.294884  ...   

c:\Users\Maria\Documents\Setups\Lib\site-packages\sklearn\utils\extmath.py:1051: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\Users\Maria\Documents\Setups\Lib\site-packages\sklearn\utils\extmath.py:1056: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
c:\Users\Maria\Documents\Setups\Lib\site-packages\sklearn\utils\extmath.py:1076: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count
C:\Users\Maria\AppData\Local\Temp\ipykernel_16856\3131174962.py:31: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  sun_hours_val = float(X_int_pol.iloc[last_obs][['sun_hours']])


,date_time,q0.025,q0.25,q0.5,q0.75,q0.975
0,2023-11-24 12:00:00,53.741995,60.603017,64.203004,67.802990,74.664012
1,2023-11-24 16:00:00,49.903376,57.099277,60.874974,64.650671,71.846573
2,2023-11-24 20:00:00,47.272065,54.629692,58.490247,62.350802,69.708429
3,2023-11-25 12:00:00,45.173038,53.576040,57.985105,62.394169,70.797171
4,2023-11-25 16:00:00,41.424830,50.103413,54.657075,59.210737,67.889321
5,2023-11-25 20:00:00,38.834939,47.648082,52.272348,56.896614,65.709757


In [ ]:
    # estimate forecast means
    mean_est = lasso.predict(X_fc)[indexes]

    # estimate forecast stds
    mean_est_hist = lasso.predict(X_pol)
    variance_est = mean_squared_error(
        y, mean_est_hist)
    indexes_shiftforw = [index+1 for index in indexes]
    forecast_std = np.array([lasso_functions.estimate_forecast_std(
        variance_est, horizon) for horizon in indexes_shiftforw])

    # estimate quantile forecasts
    quantile_forecasts = lasso_functions.get_quantiles(
        mean_est, forecast_std, quantiles)

    # return quantile forecasts in terms of absolute evaluation
    abs_eval = len(quantiles) != 6
    if abs_eval == True:
        horizon = pd.date_range(start=energydata.index[-1] + pd.DateOffset(
            hours=1), periods=periods, freq='H')
        quantile_forecasts.insert(
            0, 'date_time', [horizon[i] for i in indexes])

        return quantile_forecasts

    # else: create submission frame
    else:
        forecast_frame = create_submission_frame.get_frame(
            quantile_forecasts, indexes)
        forecast_frame = forecast_frame.drop(columns={'index'})
        horizon = pd.date_range(start=energydata.index[-1] + pd.DateOffset(
            hours=1), periods=periods, freq='H')
        forecast_frame.insert(
            0, 'date_time', [horizon[i] for i in indexes])

        return (forecast_frame)